### HTML Output Templates

In [ ]:
import json
from datetime import datetime

from bs4 import BeautifulSoup
from dateutil import parser
from jinja2 import Environment, Template


def format_date(value, format='%Y-%m-%d'):
    if value is None:
        return None

    if isinstance(value, datetime):
        return value.strftime(format)

    return parser.parse(value).strftime(format)


def format_number(value, format='{:,.2f}'):
    return format.format(value)

env = Environment()
env.filters['date'] = format_date
env.filters['number'] = format_number


def template_to_react_tables(template_string, metric_data, metric_data_historical):
    # Render the template with Jinja
    template = env.from_string(template_string)
    rendered_html = template.render(value=metric_data, metric_history=metric_data_historical)

    return parse_html_to_react_table(rendered_html)


def parse_html_to_react_table(rendered_html):
    soup = BeautifulSoup(rendered_html, 'html.parser')

    # find all `<MetricTable>` elements
    tables = soup.find_all('metrictable')
    tables_data = []

    for table in tables:
        # Extract columns
        columns = []
        headers = table.find_all('th')
        for header in headers:
            columns.append({
                "Header": header.text,
                "accessor": header.text.lower().replace(" ", "_")
            })

        # Extract data
        data = []
        rows = table.find('tbody').find_all('tr')
        for row in rows:
            row_data = {}
            for idx, cell in enumerate(row.find_all('td')):
                accessor = columns[idx]["accessor"]
                row_data[accessor] = cell.text
            data.append(row_data)

        tables_data.append((data, columns))

    return tables_data

def parse_summary_from_html(rendered_template_html):
    soup = BeautifulSoup(rendered_template_html, "html.parser")

    # find all `<table>` elements
    tables = soup.find_all("table")
    tables_data = []

    for table in tables:
        headers = [cell.text for cell in table.find_all("th")]

        tables_data.append({
            "type": "table",
            "data": [
                {
                    headers[i]: cell.text
                    for i, cell in enumerate(row.find_all("td"))
                }
                for row in table.find_all("tr")
            ],
            "metadata": {"title": ""}, # TODO: add title
        })

    return tables_data

In [ ]:
# Example usage
output_template = """
<table>
    <thead>
        <tr>
            <th>Accuracy</th>
            <th>Precision</th>
            <th>Recall</th>
            <th>F1 Score</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>{{ value["accuracy"] }}</td>
            <td>{{ value["weighted avg"]["precision"] }}</td>
            <td>{{ value["weighted avg"]["recall"] }}</td>
            <td>{{ value["weighted avg"]["f1-score"] }}</td>
        </tr>
    </tbody>
</table>
"""

In [ ]:
with open("class_perf_value.json") as f:
    metric_data = json.load(f)

In [ ]:
data, columns = template_to_react_tables(output_template, metric_data, [])[0]

print(json.dumps(data, indent=2))
print()
print(json.dumps(columns, indent=2))

In [ ]:
env.from_string(output_template).render(value=metric_data)

### Historical Data in HTML Output Template

In [ ]:
monitoring_metric_template = """
<MetricTable>
    <thead>
        <tr>
            <th>Date</th>
            <th>Accuracy</th>
            <th>Precision</th>
            <th>Recall</th>
            <th>F1 Score</th>
        </tr>
    </thead>
    <tbody>
        {% for item in metric_history %}
        <tr>
            <td>{{ item["metadata"]["created_at"] | date }}</td>
            <td>{{ item["value"]["accuracy"] | number }}</td>
            <td>{{ item["value"]["weighted avg"]["precision"] }}</td>
            <td>{{ item["value"]["weighted avg"]["recall"] }}</td>
            <td>{{ item["value"]["weighted avg"]["f1-score"] }}</td>
        </tr>
        {% endfor %}
    </tbody>
</MetricTable>
"""

In [ ]:
with open("class_perf_value_history.json") as f:
    metric_data_historical = json.load(f)

In [ ]:
tables = template_to_react_tables(monitoring_metric_template, metric_data_historical[0]["value"], metric_data_historical)

for table in tables:
    data, columns = table
    print(json.dumps(data, indent=2))
    print()
    print(json.dumps(columns, indent=2))

## JSON Summary Templates

In [ ]:
def json_template_to_summary(template, data):
    rendered = Template(template).render(value=data)
    return json.loads(rendered)

In [ ]:
simple_json_template = """
[
    {
        "type": "table",
        "data": [
            {
                "Accuracy": "{{ value['accuracy'] }}",
                "Precision": "{{ value['weighted avg']['precision'] }}",
                "Recall": "{{ value['weighted avg']['recall'] }}",
                "F1 Score": "{{ value['weighted avg']['f1-score'] }}"
            }
        ],
        "metadata": {
            "title": "Custom Classifier Performance"
        }
    }
]
"""

In [ ]:
summary = json_template_to_summary(simple_json_template, metric_data)

print(json.dumps(summary, indent=4))

In [ ]:
complex_json_template = """
[
    {
        "type": "table",
        "data": [
            {% for row in value %}
                {
                    "Bin": "{{ row['bin'] }}",
                    "PSI": "{{ row['psi'] }}"
                }{% if not loop.last %},{% endif %}
            {% endfor %}
        ],
        "metadata": {
            "title": "Custom Classifier Performance"
        }
    }
]
"""

In [ ]:
with open("psi_value.json") as f:
    metric_data = json.load(f)

In [ ]:
summary = json_template_to_summary(complex_json_template, metric_data)

print(json.dumps(summary, indent=4))

# Custom Query Language for Output Templates